In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly_express as px

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

from preprocessing import *
from feature_engineering import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
df = pd.read_csv("E-Building_Data.csv")
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(["room_number", "date_time"])

In [5]:
fe = FeatureEngineering(df) # helper class for feature enginerring
df = fe.feature_engineering(n=False)

In [6]:
# lat, long for weather station in karlsruhe
latitude = 49.0069
longitude = 8.4037
start_date = datetime(2022, 6, 2)
end_date = datetime(2023, 9, 30)
wf = WeatherFetcher(latitude, longitude, start_date, end_date) # helper class to fetch weather with api

In [7]:
df = fe.filter_rooms_by_prefix()
prefixes = ['eu', 'e0', 'e1', 'e2', 'e3']
dataframes = [df[prefix] for prefix in prefixes]


dfeu, dfe0, dfe1, dfe2, dfe3 = dataframes

In [8]:
dataframes = [dfeu, dfe0, dfe1, dfe2, dfe3]

for i in range(len(dataframes)):
    dataframes[i] = wf.combine_weather(dataframes[i])
dfeeu, dfe0, dfe1, dfe2, dfe3 = dataframes

# Apply onehotencoding for 'season' and 'room_number'
encoded_dataframes = [fe.onehotencoding(df, categorical_features=["season", "room_number"]) for df in dataframes]


dfeu, dfe0, dfe1, dfe2, dfe3 = encoded_dataframes

In [9]:
dataframes = [dfeu, dfe0, dfe1, dfe2, dfe3]

for df in dataframes:
    df.drop(['prcp', 'snow', 'wdir', 'wpgt'], axis=1, inplace=True)

In [10]:

def sliding_window_forecast(data: pd.DataFrame, label_name: str, n_in: int, n_out: int, dropna: bool = True) -> pd.DataFrame:
    """
    Transforms time series data into a supervised learning format for forecasting.

    Parameters:
    - data (pd.DataFrame): The input time series data.
    - label_name (str): The name of the target column to forecast.
    - n_in (int): Number of lag observations (input sequence length).
    - n_out (int): Number of future observations to forecast (output sequence length).
    - dropna (bool): Whether to drop rows with NaN values.

    Returns:
    - pd.DataFrame: The transformed DataFrame suitable for forecasting.
    """
    if label_name not in data.columns:
        raise ValueError(f"label_name '{label_name}' is not a column in the data")

    cols, names = list(), list()

    # Input sequence (t-n, ..., t-1, t)
    for i in range(n_in, -1, -1):
        cols.append(data.shift(i))
        names += [f"{col}(t-{i})" if i > 0 else f"{col}(t)" for col in data.columns]

    # Forecast sequence (t+1, ..., t+n)
    for i in range(1, n_out + 1):
        cols.append(data[[label_name]].shift(-i))
        names += [f"{label_name}(t+{i})"]

    
    data_reframed = pd.concat(cols, axis=1)
    data_reframed.columns = names

    if dropna:
        data_reframed.dropna(inplace=True)

    return data_reframed

In [11]:
dataframes = [dfeu,dfe0, dfe1, dfe2, dfe3]
[df.set_index("date_time", inplace=True) for df in dataframes]

[None, None, None, None, None]

In [12]:
dataframes = [dfeu, dfe0, dfe1, dfe2, dfe3]
transformed_dataframes = [sliding_window_forecast(df, "tmp", 1, 1) for df in dataframes] # forecarst the next datapoint with the previous datapoint
dfeu,dfe0, dfe1, dfe2, dfe3 = transformed_dataframes

In [13]:
def prepare_data(df):
    dropped_columns = ["tmp(t+1)", 'season(t-1)', 'room_number(t-1)', 'room_number(t)', 'season(t)']
    input_data = df.drop(dropped_columns, axis=1).values
    targets = df[["tmp(t+1)"]].values
    T = 1  # Number of timesteps to look while predicting
    D = input_data.shape[1]  # Dimensionality of the input
    N = len(input_data) - T

    # Train size: 80% of the total data size
    train_size = int(len(input_data) * 0.80)

    # Normalization of the inputs
    scaler = StandardScaler()
    scaler.fit(input_data[:train_size + T - 1])
    input_data = scaler.transform(input_data)

    # Preparing X_train and y_train
    X_train = np.zeros((train_size, T, D))
    y_train = np.zeros((train_size, 1))

    for t in range(train_size):
        X_train[t, :, :] = input_data[t:t+T]
        y_train[t] = (targets[t+T])

    # Preparing X_test and y_test
    X_test = np.zeros((N - train_size, T, D))
    y_test = np.zeros((N - train_size, 1))

    for i in range(N - train_size):
        t = i + train_size
        X_test[i, :, :] = input_data[t:t+T]
        y_test[i] = (targets[t+T])

    
    X_train = torch.from_numpy(X_train.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.float32))
    X_test = torch.from_numpy(X_test.astype(np.float32))
    y_test = torch.from_numpy(y_test.astype(np.float32))

    return X_train, y_train, X_test, y_test

In [14]:
class TempModel(nn.Module):
    def __init__(self, input_size):
        super(TempModel, self).__init__()
        self.hidden_dim_lstm = 64
        self.hidden_dim_bi_lstm = 32

        # Initial LSTM layer
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_dim_lstm, num_layers=1, batch_first=True)
        self.dropout_lstm = nn.Dropout(p=0.4)
        
        # Linear layer to match dimensions
        self.linear = nn.Linear(self.hidden_dim_lstm, 32)
        
        # Bi-LSTM Layer
        self.bi_lstm = nn.LSTM(input_size=32, hidden_size=self.hidden_dim_bi_lstm, num_layers=1, batch_first=True, bidirectional=True)
        self.dropout_bi_lstm = nn.Dropout(p=0.3)
        
        # Additional Linear layer
        self.linear1 = nn.Linear(self.hidden_dim_bi_lstm * 2, 16)  # hidden_size * 2 for bidirectional output
        
        self.dropout_lin = nn.Dropout(p=0.1)

        # Fully connected layer
        self.fc = nn.Linear(16, 1)

    def forward(self, X):
        if len(X.shape) == 2:  # Unbatched input: (sequence_length, input_dim)
            X = X.unsqueeze(0)  # Add batch dimension: (1, sequence_length, input_dim)

        batch_size = X.size(0)

        # Initialize hidden state and cell state with zeros for LSTM and Bi-LSTM
        h0 = torch.zeros(1, batch_size, self.hidden_dim_lstm).to(X.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(1, batch_size, self.hidden_dim_lstm).to(X.device)  # (num_layers, batch_size, hidden_dim)
        h0_bi = torch.zeros(2, batch_size, self.hidden_dim_bi_lstm).to(X.device)  # (num_layers*2, batch_size, hidden_dim) for bidirectional
        c0_bi = torch.zeros(2, batch_size, self.hidden_dim_bi_lstm).to(X.device)  # (num_layers*2, batch_size, hidden_dim) for bidirectional

        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(X, (h0, c0))

        out = self.dropout_lstm(out[:, -1, :])  # Apply dropout to the last time step's output

        # Apply linear layer to match dimensions
        out = self.linear(out)

        # Reshape for Bi-LSTM
        out = out.unsqueeze(1)

        # Forward propagate Bi-LSTM
        out, (hn_bi, cn_bi) = self.bi_lstm(out, (h0_bi, c0_bi))

        # Apply dropout to the output of the Bi-LSTM
        out = self.dropout_bi_lstm(out[:, -1, :])  # Apply dropout to the last time step's output

        # Apply another linear layer
        out = self.linear1(out)

        # Apply final dropout
        out = self.dropout_lin(out)

        # Fully connected layer
        out = self.fc(out)
        
        return out


### Uncomment this cell, if u want to train the models takes about 60min for all 4 models.

In [15]:
# def train(model, learning_rate, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):

    
#     criterion = torch.nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
#     train_losses = []
#     test_losses = []
    
    
#     train_dataset = TensorDataset(X_train, y_train)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    
#     test_dataset = TensorDataset(X_test, y_test)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
#     for epoch in range(epochs):
#         model.train()
        
#         epoch_train_loss = 0.0
#         for batch_X, batch_y in train_loader:
#             optimizer.zero_grad()
            
#             # Forward pass
#             outputs = model(batch_X)
#             loss = criterion(outputs, batch_y)
            
#             # Backward pass and optimization
#             loss.backward()
#             optimizer.step()
            
#             epoch_train_loss += loss.item()
        
#         # Compute average training loss for the epoch
#         train_loss = epoch_train_loss / len(train_loader)
#         train_losses.append(train_loss)
        
#         # Compute test loss
#         model.eval()
#         epoch_test_loss = 0.0
#         with torch.no_grad():
#             for batch_X_test, batch_y_test in test_loader:
#                 test_outputs = model(batch_X_test)
#                 loss = criterion(test_outputs, batch_y_test)
#                 epoch_test_loss += loss.item()
        
#         # Compute average test loss for the epoch
#         test_loss = epoch_test_loss / len(test_loader)
#         test_losses.append(test_loss)
        
#         if (epoch + 1) % 10 == 0 or epoch == 0:
#             print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Using device: {device}')
    
#     return train_losses, test_losses



# floor_dfs = [dfeu, dfe0, dfe1, dfe2, dfe3]
# for i, df in enumerate(floor_dfs, start=0):
#     X_train, y_train, X_test, y_test = prepare_data(df)
#     learning_rate = 0.001
#     epochs = 100
#     batch_size = 32
#     input_size = X_train.shape[2]
#     model = TempModel(input_size)
#     train_losses, test_losses = train(model=model, learning_rate=learning_rate, X_train=X_train, 
#                                   y_train=y_train, X_test=X_test, y_test=y_test, epochs=epochs, batch_size=batch_size)
    
#     # Save the trained model and losses
#     torch.save(model.state_dict(), f'floor{i}_lstm_model.pth')
#     with open(f'floor{i}_losses.txt', 'w') as file:
#         file.write("Train Losses:\n")
#         for loss in train_losses:
#             file.write(f"{loss}\n")
#         file.write("\nTest Losses:\n")
#         for loss in test_losses:
#             file.write(f"{loss}\n")
    
#     print(f'Model for floor {i} trained, saved, and losses recorded.')

In [16]:
# List of dataframes
dataframes = [dfeu,dfe0, dfe1, dfe2, dfe3]

# Process each dataframe and print the shape
input_data_list = []
for i, df in enumerate(dataframes):
    dropped_columns = ["tmp(t+1)", 'season(t-1)', 'room_number(t-1)', 'room_number(t)', 'season(t)']
    input_data = df.drop(dropped_columns, axis=1).values
    input_data_list.append(input_data)
    print(f"Shape of input_data{i+1}: {input_data.shape}") # depending of number of rooms in each floor

Shape of input_data1: (1682, 42)
Shape of input_data2: (41837, 50)
Shape of input_data3: (54996, 54)
Shape of input_data4: (32914, 50)
Shape of input_data5: (53665, 54)


In [17]:
model_configs = [
    {'input_size': 42, 'path': r".\trained_models\floor0_lstm_model.pth"},
    {'input_size': 50, 'path': r".\trained_models\floor1_lstm_model.pth"},
    {'input_size': 54, 'path': r".\trained_models\floor2_lstm_model.pth"},
    {'input_size': 50, 'path': r".\trained_models\floor3_lstm_model.pth"},
    {'input_size': 54, 'path': r".\trained_models\floor4_lstm_model.pth"},
]

models = []

for config in model_configs:
    model = TempModel(input_size=config['input_size'])
    model.load_state_dict(torch.load(config['path']))
    model.eval()
    models.append(model)

In [18]:
dataframes = [dfeu, dfe0, dfe1, dfe2, dfe3]
directory = "trained_models"

for i, df in enumerate(dataframes, start=0):
    # Construct the file path
    file_path = os.path.join(directory, f'floor{i}_losses.txt')
    
    # Load losses from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    train_losses = []
    test_losses = []
    is_test_loss = False
    
    for line in lines:
        if line.strip() == 'Train Losses:':
            is_test_loss = False
            continue
        elif line.strip() == 'Test Losses:':
            is_test_loss = True
            continue
        elif line.strip() == '':
            continue
        
        if not is_test_loss:
            train_losses.append(float(line.strip()))
        else:
            test_losses.append(float(line.strip()))
        
    epochs = list(range(1, len(train_losses) + 1))
    loss_data = {'Epochs': epochs, 'Train Loss': train_losses, 'Test Loss': test_losses}
    loss_df = pd.DataFrame(loss_data)
    
    # Plot using Plotly Graph Objects
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=loss_df['Epochs'], y=loss_df['Train Loss'], mode='lines', name='Train Loss'))
    fig.add_trace(go.Scatter(x=loss_df['Epochs'], y=loss_df['Test Loss'], mode='lines', name='Test Loss'))
    
    fig.update_layout(title=f'Loss for Floor {i}', xaxis_title='Epochs', yaxis_title='Loss')
    
    fig.show()

In [23]:
len(models)

5

In [21]:
model0, model1, model2, model3, model4 = models[:5]

In [36]:
models = [model1, model2, model3, model4]
dataframes = [dfe0, dfe1, dfe2, dfe3]

results = {
    'Floor 0 True Values': [],
    'Floor 1 True Values': [],
    'Floor 2 True Values': [],
    'Floor 3 True Values': [],
    'Floor 0 Predictions': [],
    'Floor 1 Predictions': [],
    'Floor 2 Predictions': [],
    'Floor 3 Predictions': []
}

# Determine device once
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Iterate over each model and dataframe
for model_name, (df, model) in zip(['Floor 0', 'Floor 1', 'Floor 2', 'Floor 3'], zip(dataframes, models)):
    X_train, y_train, X_test, y_test = prepare_data(df)

    # Move model and data to the device
    model.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    # Set model to evaluation mode
    model.eval()

    # Perform prediction
    with torch.no_grad():
        y_pred = model(X_test)

    y_pred = y_pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    # Update the results dictionary
    results[f'{model_name} True Values'].extend(y_test.flatten())
    results[f'{model_name} Predictions'].extend(y_pred.flatten())

# Debug statement to check lengths of arrays in the results dictionary
print("Lengths of arrays in the results dictionary:")
for key, value in results.items():
    print(f"{key}: {len(value)}")

# Ensure all arrays have the same length before creating DataFrame
min_length = min(len(value) for value in results.values())

# Trim all arrays to the same length
for key in results:
    results[key] = results[key][:min_length]

# Convert results dictionary to DataFrame
df_results = pd.DataFrame(results)

# Display the resulting DataFrame
df_results

Lengths of arrays in the results dictionary:
Floor 0 True Values: 8367
Floor 1 True Values: 10999
Floor 2 True Values: 6582
Floor 3 True Values: 10732
Floor 0 Predictions: 8367
Floor 1 Predictions: 10999
Floor 2 Predictions: 6582
Floor 3 Predictions: 10732


,Floor 0 True Values,Floor 1 True Values,Floor 2 True Values,Floor 3 True Values,Floor 0 Predictions,Floor 1 Predictions,Floor 2 Predictions,Floor 3 Predictions
0,26.052856,22.787500,24.000000,25.083750,26.077080,22.865433,24.006418,23.113991
1,26.018000,22.742500,23.966667,23.932632,26.074474,22.827139,23.989676,24.074505
2,25.391111,22.712500,23.620001,23.131666,26.030003,22.713940,23.990458,25.938192
3,25.491539,22.680000,23.709999,23.682352,25.995022,22.787704,24.021461,23.483994
4,25.813334,22.645000,23.932501,23.826000,25.645901,22.792221,24.196775,23.700459
...,...,...,...,...,...,...,...,...
6577,24.850000,20.934999,24.195000,18.162500,25.799206,21.409468,24.629992,18.397726
6578,24.870001,20.747499,23.722500,18.100000,26.004011,21.961502,24.281546,18.196461
6579,24.895000,20.660000,23.320000,18.065001,26.122974,21.289286,23.899496,18.107864
6580,24.912500,20.613333,22.992500,18.007500,26.201220,21.055258,23.524822,18.022259


In [39]:
models = [model1, model2, model3, model4]
dataframes = [dfe0, dfe1, dfe2, dfe3]

figures = []

# Iterate over each model and dataframe
for floor, (df, model) in enumerate(zip(dataframes, models)):
    
    X_train, y_train, X_test, y_test = prepare_data(df)

    # Set model to evaluation mode
    model.eval()

    # Perform prediction
    with torch.no_grad():
        if torch.cuda.is_available():
            X_test = X_test.cuda()
        y_pred = model(X_test)

    if torch.cuda.is_available():
        y_pred = y_pred.cpu().numpy()
        y_test = y_test.cpu().numpy()
    else:
        y_pred = y_pred.numpy()
        y_test = y_test.numpy()

    # Create a DataFrame with true values and predicted values
    df_results = pd.DataFrame({
        'Index': range(len(y_test)),
        'True Values': y_test.flatten(),
        'Predicted Values': y_pred.flatten()
    })

    df_melted = df_results.melt(id_vars=['Index'], 
                                value_vars=['True Values', 'Predicted Values'],
                                var_name='Type', value_name='Value')
    
    fig = go.Figure()
    
    # Add traces for True Values and Predictions
    for value_type in df_melted['Type'].unique():
        filtered_df = df_melted[df_melted['Type'] == value_type]
        fig.add_trace(go.Scatter(x=filtered_df['Index'], y=filtered_df['Value'], mode='lines', name=value_type))
    
    fig.update_layout(title=f'Etage {floor}: Vorhersage von t+1 mit t-1 and t', xaxis_title='Index', yaxis_title='Temperatur °C')
    
    figures.append(fig)

# Show figures
for fig in figures:
    fig.show()

In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

models = [model0, model1, model2, model3, model4]
dataframes = [dfeu, dfe0, dfe1, dfe2, dfe3]

results = {
    'Floor EU True Values': [],
    'Floor 0 True Values': [],
    'Floor 1 True Values': [],
    'Floor 2 True Values': [],
    'Floor 3 True Values': [],
    'Floor EU Predictions': [],
    'Floor 0 Predictions': [],
    'Floor 1 Predictions': [],
    'Floor 2 Predictions': [],
    'Floor 3 Predictions': []
}

metrics = {
    'Floor EU': {'RMSE': None, 'MAE': None},
    'Floor 0': {'RMSE': None, 'MAE': None},
    'Floor 1': {'RMSE': None, 'MAE': None},
    'Floor 2': {'RMSE': None, 'MAE': None},
    'Floor 3': {'RMSE': None, 'MAE': None},
}

# Determine device once
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Iterate over each model and dataframe
for model_name, (df, model) in zip(['Floor EU', 'Floor 0', 'Floor 1', 'Floor 2', 'Floor 3'], zip(dataframes, models)):
    X_train, y_train, X_test, y_test = prepare_data(df)

    # Move model and data to the device
    model.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    # Set model to evaluation mode
    model.eval()

    # Perform prediction
    with torch.no_grad():
        y_pred = model(X_test)

    y_pred = y_pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    # Calculate RMSE and MAE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Store the metrics
    metrics[model_name]['RMSE'] = rmse
    metrics[model_name]['MAE'] = mae

    # Update the results dictionary
    results[f'{model_name} True Values'].extend(y_test.flatten())
    results[f'{model_name} Predictions'].extend(y_pred.flatten())

# Print RMSE and MAE for each model
for model_name in metrics:
    print(f"{model_name} - RMSE: {metrics[model_name]['RMSE']:.4f}, MAE: {metrics[model_name]['MAE']:.4f}")

Floor EU - RMSE: 0.4364, MAE: 0.3644
Floor 0 - RMSE: 0.9576, MAE: 0.7732
Floor 1 - RMSE: 0.7594, MAE: 0.6088
Floor 2 - RMSE: 0.6095, MAE: 0.4567
Floor 3 - RMSE: 0.9275, MAE: 0.7233
